In [ ]:
from tensorflow.keras.applications import VGG19, DenseNet201
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout,Conv2D, BatchNormalization,Activation, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
import tensorflow as tf
import cv2 
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import eli5
from eli5.sklearn import PermutationImportance
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
from skimage.io import imread
from skimage.transform import resize
from skimage.segmentation import mark_boundaries
import os
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import pprint
from collections import deque
import copy
from tensorflow.keras.callbacks import ReduceLROnPlateau
import numpy as np

In [ ]:
base_path = "/kaggle/input/chest-xray-covid19-pneumonia"
categories = ['NORMAL', 'COVID19', 'PNEUMONIA']
image_size = (224,224)

In [ ]:
def load_images_from_folder(folder, label_name, images_list, labels_list):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        img = cv2.imread(file_path)
        if img is not None:
            img = cv2.resize(img, image_size)
            images_list.append(img)
            labels_list.append(label_name)

In [ ]:
# Load training data
train_path = os.path.join(base_path, 'Data', 'train')
train_images, train_labels = [], []

for category in categories:
    category_path = os.path.join(train_path, category)
    print(f"Loading training data from {category_path}...")
    load_images_from_folder(category_path, category, train_images, train_labels)

In [ ]:
# Load test data
test_path = os.path.join(base_path, 'Data', 'test')
test_images, test_labels = [], []

for category in categories:
    category_path = os.path.join(test_path, category)
    print(f"Loading test data from {category_path}...")
    load_images_from_folder(category_path, category, test_images, test_labels)

In [ ]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
from sklearn.utils import shuffle

# Shuffle training data
train_images, train_labels = shuffle(train_images, train_labels, random_state=42)

# Shuffle test data
test_images, test_labels = shuffle(test_images, test_labels, random_state=42)

In [ ]:
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

In [ ]:
label_mapping = {'COVID19': 0, 'PNEUMONIA': 1,'NORMAL': 2}

train_labels_int = np.array([label_mapping[label] for label in train_labels], dtype='int32')
test_labels_int = np.array([label_mapping[label] for label in test_labels], dtype='int32')

train_labels_one_hot = tf.keras.utils.to_categorical(train_labels_int, num_classes=3).astype('float32')
test_labels_one_hot = tf.keras.utils.to_categorical(test_labels_int, num_classes=3).astype('float32')

In [ ]:
train_images.shape

In [ ]:
test_images.shape

# optimizing COVIDet's hyperparameters using ant colony 

In [ ]:
def build_model_fixed(img_shape, num_classes):
    inputs = Input(shape=img_shape)

    x = Conv2D(32, (3, 3), padding='same')(inputs) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    for i in range(2): 
        x = Conv2D(64, (3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)

    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x) 
    x = Dense(64, activation='relu')(x) 
    
    outputs = Dense(num_classes, activation='softmax')(x) 

    model = Model(inputs=inputs, outputs=outputs)
    
 
    optimizer = Adam(learning_rate=0.001) 
    
    model.compile(
        optimizer=optimizer, 
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
        ]
    )
    return model

In [ ]:
img_shape = train_images.shape[1:] 
num_classes = 3


model = build_model_fixed(img_shape, num_classes)


history = model.fit(
    x=train_images,
    y=train_labels_one_hot,
    validation_data=(test_images, test_labels_one_hot),
    epochs=15,        
    batch_size=32,      
    verbose=1
)

results = model.evaluate(test_images, test_labels_one_hot, verbose=1)


print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")

In [ ]:
import numpy as np
import math
import copy
from lime import lime_image


# Bounds: [num_samples (100-2000), kernel_width (0.1-3.0)]
LOWER_BOUND = np.array([100, 0.1])
UPPER_BOUND = np.array([2000, 3.0])
def predict_wrapper(images):
        images = images.astype('float32')
        return model.predict(images, verbose=0)
    
def evaluate_lime_fidelity(params, model, image, original_label):
    n_samples = int(params[0])
    k_width = params[1]
    n_samples = max(10, n_samples) 
    k_width = max(0.01, k_width)

    explainer = lime_image.LimeImageExplainer(kernel_width=k_width, verbose=False)

    explanation = explainer.explain_instance(
        image.astype('double'), 
        predict_wrapper, 
        top_labels=1, 
        hide_color=0, 
        num_samples=n_samples,
        random_seed=42
    )
    
    temp, mask = explanation.get_image_and_mask(
        original_label, positive_only=True, num_features=5, hide_rest=False
    )
    

    perturbed_image = copy.deepcopy(image)
    perturbed_image[mask == 1] = 0 
    

    preds_orig = predict_wrapper(image[np.newaxis, ...])[0][original_label]
    preds_pert = predict_wrapper(perturbed_image[np.newaxis, ...])[0][original_label]
    
    mask_coverage = np.sum(mask) / mask.size
    
    print(f"  [Debug] Orig: {preds_orig:.4f} | Pert: {preds_pert:.4f} | Mask Cover: {mask_coverage:.2%}")
    
    return preds_orig - preds_pert



In [ ]:
def levy_flight(beta=1.5):
    sigma = (math.gamma(1 + beta) * math.sin(math.pi * beta / 2) / 
             (math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
    u = np.random.normal(0, sigma, size=2)
    v = np.random.normal(0, 1, size=2)
    step = u / abs(v) ** (1 / beta)
    return step

def optimize_lime_cuckoo(model, image, label, n_nests=10, max_iter=5, pa=0.25):
    print("Starting Cuckoo Search for LIME parameters...")
    
    nests = np.random.uniform(LOWER_BOUND, UPPER_BOUND, (n_nests, 2))
    fitness = np.zeros(n_nests)
    
    for i in range(n_nests):
        fitness[i] = evaluate_lime_fidelity(nests[i], model, image, label)
        
    best_idx = np.argmax(fitness)
    best_nest = nests[best_idx].copy()
    best_score = fitness[best_idx]
    
    for t in range(max_iter):
        print(f"Iter {t+1}/{max_iter} | Best Score: {best_score:.4f} | Params: {best_nest}")
        
        new_nests = nests.copy()
        for i in range(n_nests):
            step_size = 0.01 * levy_flight() * (nests[i] - best_nest)
            new_nests[i] += step_size * np.random.randn(2)
            
            new_nests[i] = np.clip(new_nests[i], LOWER_BOUND, UPPER_BOUND)

        for i in range(n_nests):
            f_new = evaluate_lime_fidelity(new_nests[i], model, image, label)
            if f_new > fitness[i]:
                fitness[i] = f_new
                nests[i] = new_nests[i]
 
        sorted_indices = np.argsort(fitness)
        n_abandon = int(n_nests * pa)
        
        for i in range(n_abandon):
            idx = sorted_indices[i] # The worst indices
            nests[idx] = np.random.uniform(LOWER_BOUND, UPPER_BOUND, 2)
            fitness[idx] = evaluate_lime_fidelity(nests[idx], model, image, label)
            

        current_best_idx = np.argmax(fitness)
        if fitness[current_best_idx] > best_score:
            best_score = fitness[current_best_idx]
            best_nest = nests[current_best_idx].copy()
            
    return best_nest, best_score

In [ ]:
target_img = test_images[0]
target_label = np.argmax(test_labels_one_hot[0])

best_params, best_fidelity = optimize_lime_cuckoo(model, target_img, target_label)

print("\nOPTIMIZATION FINISHED")
print(f"Optimal Samples: {int(best_params[0])}")
print(f"Optimal Kernel Width: {best_params[1]:.4f}")

In [ ]:
default_samples = 1000
default_width = 0.25

opt_samples = int(best_params[0])
opt_width = best_params[1]

print("Generating 'Before' (Default) Explanation...")
explainer_default = lime_image.LimeImageExplainer(kernel_width=default_width, verbose=False)
exp_default = explainer_default.explain_instance(
    target_img.astype('double'), predict_wrapper, top_labels=1, hide_color=0, 
    num_samples=default_samples, random_seed=42
)

print("Generating 'After' (Optimized) Explanation...")
explainer_opt = lime_image.LimeImageExplainer(kernel_width=opt_width, verbose=False)
exp_opt = explainer_opt.explain_instance(
    target_img.astype('double'), predict_wrapper, top_labels=1, hide_color=0, 
    num_samples=opt_samples, random_seed=42
)

pred_class = np.argmax(model.predict(target_img[np.newaxis, ...])[0])
temp_def, mask_def = exp_default.get_image_and_mask(
    pred_class, positive_only=True, num_features=5, hide_rest=False
)
temp_opt, mask_opt = exp_opt.get_image_and_mask(
    pred_class, positive_only=True, num_features=5, hide_rest=False
)


fig, ax = plt.subplots(1, 3, figsize=(18, 6))

ax[0].imshow(target_img)
ax[0].set_title(f"Original Image\n(Class: {categories[pred_class]})", fontsize=14)
ax[0].axis('off')

ax[1].imshow(mark_boundaries(temp_def / 2 + 0.5, mask_def))
ax[1].set_title(f"Before Optimization (Default)\nS={default_samples}, W={default_width}", fontsize=14)
ax[1].axis('off')

ax[2].imshow(mark_boundaries(temp_opt / 2 + 0.5, mask_opt))
ax[2].set_title(f"After Optimization (Cuckoo)\nS={opt_samples}, W={opt_width:.2f}", fontsize=14, color='green', fontweight='bold')
ax[2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

def optimize_lime_bat(model, image, label, n_bats=10, max_iter=5):
    print("Starting Bat Algorithm (BA) optimization...")

    f_min = 0.0
    f_max = 2.0
    

    A = 0.9      # Initial Loudness 
    r = 0.1      # Initial Pulse Rate 
    alpha = 0.9  # Cooling factor for Loudness
    gamma = 0.9  # Warming factor for Pulse Rate
    
    positions = np.random.uniform(LOWER_BOUND, UPPER_BOUND, (n_bats, 2))
    velocities = np.zeros((n_bats, 2))
    fitness = np.zeros(n_bats)
    
    for i in range(n_bats):
        fitness[i] = evaluate_lime_fidelity(positions[i], model, image, label)
    

    best_idx = np.argmax(fitness)
    best_pos = positions[best_idx].copy()
    best_score = fitness[best_idx]

    for t in range(max_iter):
        print(f"Iter {t+1}/{max_iter} | Best Score: {best_score:.4f} | Params: {best_pos}")
        
        for i in range(n_bats):
            beta = np.random.rand()
            freq = f_min + (f_max - f_min) * beta
            
            velocities[i] += (positions[i] - best_pos) * freq
            
            new_pos = positions[i] + velocities[i] 
            new_pos = np.clip(new_pos, LOWER_BOUND, UPPER_BOUND)

            if np.random.rand() > r:
                epsilon = np.random.uniform(-1, 1)
                new_pos = best_pos + epsilon * A
                new_pos = np.clip(new_pos, LOWER_BOUND, UPPER_BOUND)
            
            new_score = evaluate_lime_fidelity(new_pos, model, image, label)
            

            if (new_score > fitness[i]) and (np.random.rand() < A):
                positions[i] = new_pos
                fitness[i] = new_score
                
                A *= alpha      
                r *= (1 - np.exp(-gamma * t)) 
                
            if new_score > best_score:
                best_score = new_score
                best_pos = new_pos.copy()
                
    return best_pos, best_score


bat_params, bat_score = optimize_lime_bat(model, target_img, target_label)

print("\nBAT OPTIMIZATION FINISHED")
print(f"Optimal Samples: {int(bat_params[0])}")
print(f"Optimal Kernel Width: {bat_params[1]:.4f}")

In [ ]:
opt_samples_bat = int(bat_params[0])
opt_width_bat = bat_params[1]

print("Generating Default Explanation...")
explainer_default = lime_image.LimeImageExplainer(kernel_width=default_width, verbose=False)
exp_default = explainer_default.explain_instance(
    target_img.astype('double'), predict_wrapper, top_labels=1, hide_color=0, 
    num_samples=default_samples, random_seed=42
)


print("Generating Bat Optimized Explanation...")
explainer_bat = lime_image.LimeImageExplainer(kernel_width=opt_width_bat, verbose=False)
exp_bat = explainer_bat.explain_instance(
    target_img.astype('double'), predict_wrapper, top_labels=1, hide_color=0, 
    num_samples=opt_samples_bat, random_seed=42
)

pred_class = np.argmax(model.predict(target_img[np.newaxis, ...])[0])

temp_def, mask_def = exp_default.get_image_and_mask(
    pred_class, positive_only=True, num_features=5, hide_rest=False
)

temp_bat, mask_bat = exp_bat.get_image_and_mask(
    pred_class, positive_only=True, num_features=5, hide_rest=False
)


fig, ax = plt.subplots(1, 3, figsize=(18, 6))

# Plot 1: Original
ax[0].imshow(target_img)
label_name = categories[pred_class] if 'categories' in globals() else str(pred_class)
ax[0].set_title(f"Original Image\n(Class: {label_name})", fontsize=14)
ax[0].axis('off')

# Plot 2: Default LIME
ax[1].imshow(mark_boundaries(temp_def / 2 + 0.5, mask_def))
ax[1].set_title(f"Default LIME (Unoptimized)\nS={default_samples}, W={default_width}", fontsize=14)
ax[1].axis('off')

# Plot 3: Bat Optimized LIME
ax[2].imshow(mark_boundaries(temp_bat / 2 + 0.5, mask_bat))
ax[2].set_title(f"Bat Algorithm Optimized\nS={opt_samples_bat}, W={opt_width_bat:.2f}", fontsize=14, color='purple', fontweight='bold')
ax[2].axis('off')

plt.tight_layout()
plt.show()